In [ ]:
import cv2
from hand_tracker import HandTracker
import os
import json

In [ ]:
WINDOW = "Hand Tracking"
PALM_MODEL_PATH = "./palm_detection_without_custom_op.tflite"
LANDMARK_MODEL_PATH = "./hand_landmark.tflite"
ANCHORS_PATH = "./anchors.csv"

POINT_COLOR = (0, 255, 0)
CONNECTION_COLOR = (255, 0, 0)
THICKNESS = 2

In [ ]:
detector = HandTracker(
    PALM_MODEL_PATH,
    LANDMARK_MODEL_PATH,
    ANCHORS_PATH,
    box_shift=0.2,
    box_enlarge=1.3
)

In [ ]:
def saveImage(frame, count):
    to = "./dataset/images/"
    name = str(count)
    name = name.rjust(5, '0')
    name += '.jpg'
    print(name)
    count += 1
    cv2.imwrite(to+name, frame)
    return count, name

In [ ]:
#        8   12  16  20
#        |   |   |   |
#        7   11  15  19
#    4   |   |   |   |
#    |   6   10  14  18
#    3   |   |   |   |
#    |   5---9---13--17
#    2    \         /
#     \    \       /
#      1    \     /
#       \    \   /
#        ------0-

tips = [4, 8, 12, 16, 20]

In [ ]:
def makeDataset():
    k = 0
    cv2.namedWindow(WINDOW)
    capture = cv2.VideoCapture(0)

    if capture.isOpened():
        hasFrame, frame = capture.read()
    else:
        hasFrame = False
    while hasFrame:
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        points, _ = detector(image)

        dispFrame = frame.copy()
        label = []
        if points is not None:
            k += 1
            for i in tips:
                x, y = points[i]
                x = int(x)
                y = int(y)
                label.append((x, y))
                cv2.circle(dispFrame, (x, y), THICKNESS *
                           2, POINT_COLOR, THICKNESS)


        cv2.imshow(WINDOW, dispFrame)
        if points is not None:
            name = 'aa'
            count, name = saveImage(frame, count)
            writeToJson(name, label)
        hasFrame, frame = capture.read()
        key = cv2.waitKey(20)
        if k == 3:
            break
        if key == 113:
            break
    capture.release()
    cv2.destroyAllWindows()

In [ ]:
labelName = "./dataset/floatLabels.json"
newobj = json.dumps(newlabels, indent=4)
with open(labelName, "w") as outfile:
    outfile.write(newobj)

In [ ]:
datasetPath = '/home/mufaddal/Mufaddal/Programs/CNN_learning/hand_tracking/dataset/'

In [ ]:
import json
import cv2
import os
import numpy as np

In [ ]:
def getImageNames(path):
    imageNameList = []
    for r,_,f in os.walk(path):
        for imgName in f:
           if imgName.endswith(".jpg"):
              imageNameList.append(imgName)
    return sorted(imageNameList)

In [ ]:
def randomizeDataset(path):
    floatLabelPath = path + 'floatLabels.json'
    intLabelPath = path + 'labels.json'
    imagePath = path + 'images/'
    
    labels = json.load(open(intLabelPath, 'r'))
    
    imageNameList = getImageNames(imagePath)
    repList = []
    for i in range(1000):
        index1 = np.random.randint(int(4100))
        index2 = np.random.randint(int(4100))
        
        i1path = imagePath+imageNameList[index1]
        i2path = imagePath+imageNameList[index2]
        repList.append((imageNameList[index1], imageNameList[index2]))
        img1 = cv2.imread(i1path)
        img2 = cv2.imread(i2path)
        
        #img1, img2 = img2, img1
        
        label1 = labels[imageNameList[index1]]
        label2 = labels[imageNameList[index2]]
        
        #label1, label2 = label2, label1
        
        labels[imageNameList[index1]] = label1
        labels[imageNameList[index2]] = label2
        
        #os.remove(i1path)
        #os.remove(i2path)
        
        #cv2.imwrite(i1path, img1)
        #cv2.imwrite(i2path, img2)
        
   
    #newobj = json.dumps(labels, indent=4)
    #with open(intLabelPath, "w") as outfile:
        #outfile.write(newobj)
    return repList

In [ ]:
def saveJson(path, jsonData):
    newobj = json.dumps(jsonData, indent=4)
    with open(path, "w") as outfile:
        outfile.write(newobj)

In [ ]:
img_row = 480
img_col = 640

In [ ]:
def makeFloatLabels(path):
    intLabelPath = path+ 'labels.json'
    floatLabelPath = path + 'floatLabels.json'
    intLabel = json.load(open(intLabelPath, 'r'))
    floatLabel = {}
    for label in intLabel:
        #print(label)
        #print(intLabel[label])
        nlabel = []
        for box in intLabel[label]:
            x,y = box
            x = float(x/img_col)
            y = float(y/img_row)
            if x>=0 and x<=1 and y>= 0 and y<=1:
                nlabel.append((x,y))
            else:
                print(label, x,y)
            #print(x, y)
        floatLabel[label] = nlabel
    saveJson(floatLabelPath, floatLabel)

In [ ]:
makeFloatLabels(datasetPath)

In [ ]:
def makeFile(path):
    labelPath = path + 'floatLabels.json'
    imagePath = path + 'images/'
    
    ht = float(35/img_row)
    wd = float(35/img_col)
    
    imageNameList = getImageNames(imagePath)
    labels = json.load(open(labelPath, 'r'))
    paths = []
    print(labels)
    for name in imageNameList:
        label = ''
        imgpath = imagePath + name
        
        if name in labels.keys():
            paths.append(imgpath)
            points = labels[name]
            for point in points:
                x,y = point
                #print(x,y)
                x = str(x)
                y = str(y)
                label += (' '.join(['0', x, y, str(ht), str(wd)]))
                label += '\n'
            imgpath = imgpath.split('.')[0] + '.txt'
            print(imgpath)
            imglabel = open(imgpath, 'w')
            imglabel.write(label)
            imglabel.close()
    return paths

In [ ]:
lines = makeFile(datasetPath)

In [ ]:
import os
import json
import cv2

In [ ]:
datasetPath = '/content/dataset/'

In [ ]:
img_row = 480
img_col = 640

In [ ]:
def getImageNames(path):
    imageNameList = []
    for r,_,f in os.walk(path):
        for imgName in f:
           if imgName.endswith(".jpg"):
              imageNameList.append(imgName)
    return sorted(imageNameList)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def saveJson(path, jsonData):
    newobj = json.dumps(jsonData, indent=4)
    with open(path, "w") as outfile:
        outfile.write(newobj)

In [ ]:
def makeFloatLabels(path):
    intLabelPath = path+ 'labels.json'
    floatLabelPath = path + 'floatLabels.json'
    intLabel = json.load(open(intLabelPath, 'r'))
    floatLabel = {}
    for label in intLabel:
        #print(label)
        #print(intLabel[label])
        nlabel = []
        for box in intLabel[label]:
            x,y = box
            x = float(x/img_col)
            y = float(y/img_row)
            if x>=0 and x<=1 and y>= 0 and y<=1:
                nlabel.append((x,y))
            else:
                print(label, x,y)
            #print(x, y)
        floatLabel[label] = nlabel
    saveJson(floatLabelPath, floatLabel)

In [ ]:
def makeFiles(path):
    labelPath = path + 'floatLabels.json'
    imagePath = path + 'images/'
    
    ht = float(35/img_row)
    wd = float(35/img_col)
    
    imageNameList = getImageNames(imagePath)
    labels = json.load(open(labelPath, 'r'))
    paths = []
    print(labels)
    for name in imageNameList:
        label = ''
        imgpath = imagePath + name
        
        if name in labels.keys():
            paths.append(imgpath)
            points = labels[name]
            for point in points:
                x,y = point
                #print(x,y)
                x = str(x)
                y = str(y)
                label += (' '.join(['0', x, y, str(ht), str(wd)]))
                label += '\n'
            imgpath = imgpath.split('.')[0] + '.txt'
            print(imgpath)
            imglabel = open(imgpath, 'w')
            imglabel.write(label)
            imglabel.close()
    return paths

In [ ]:
lines = makeFiles(datasetPath)